In [0]:
######################################~15 mins
#functions that begin with "fn." are custom r functions
##########################
#numerators section;
# creates new R dataframes with "numerators" of per share columns, multiplying per-share data by appropriate share count (primary shares or fully-diluted shares)
#a newer and maybe better alternative to R dataframes are R datatables
#if i was starting over from scratch in R, I might use datatables rather than dataframes
#I am unsure what is best to use in Python, datatable or dataframe or something else
#ch <- odbcConnect("xpressfeed", uid = "xpressfeed", pwd = "xpressfeed") #old code used to access SQL
setwd(r_ri.num) #sets the directory on my computer that data is accessed from and saved to
######################
rm(ch) #removes the ch “vector”# this code is probably not needed

#inputs
mypers.i = c(-1:0) #pers means “periods”; in this case,a period is a year
mypers <- ifelse(mypers.i >=0, mypers.i, paste('m', abs(mypers.i), sep='') ) #"m" mean "minus"; m1 means "minus1" or "-1"

#get fully diluted per share and primary pershare colnums using mydata_af.0
fn.flga('mydata_af.0') #load, get, assign R file named 'mydata_af.0' (sorts the same as "File/Open" in ecel)
fn.flga('myshsinfo') #this is an R list #load, get, assign R file  named 'myshsinfo'

#loads data files using the custom R-function “fn.flga”
#fn.flga('mycols_fd') #columns with "fully diluted" per share data 
#fn.flga('mycols_pr') #columns with "primary" per share data 
#fn.flga('myunsure_ps') #columns with per share data where we are uncertain if the data is "primary AKA primary" or "fully diluted" per share data 

mycols_fd <- match(myshsinfo$co_afnd_fd$columnname, colnames(mydata_af.0) ) #an array of column numbers that have full diluted per share data
mycols_pr <- match (c(myshsinfo$co_afnd_pr$columnname,myshsinfo$co_afnd_unsure$columnname) , colnames(mydata_af.0) ) #an array of column numbers that have primary per share data

#Create df with total assets in non share cols, assets per primary share in  per primary share cols, and assets per fully diluted share in fully diluted share cols?
#Which cols are per primary share cols? mycols_pr
#Which cols are per fully diluted share cols? mycols_fd
#Which cols are not share cols? c(-mycol_fdshs, -mycol_prshs)

coltype <- 1:ncol(mydata_af.0) #creates an array whose length equals the number of columns in the mydata_af.0 dataframe
coltype [mycols_pr] <- 'p' #primary columns #assigns the value "p" to the primary columns
coltype [mycols_fd] <- 'f' #fully diluted columns 
coltype [c(-mycols_fd, -mycols_pr)] <- 't' #t means "total", rather than per-share
dim(mydata_af.0)
#checks
summary(is.na(coltype)) 
summary(coltype=='p'|coltype=='f'|coltype=='t')
summary(coltype=='p') 
summary(coltype=='f') 
summary(coltype=='t')
fn.asr('coltype') #use this later for calculating denominator of roa df's (that is, a df with: assets per primary share, assets per fully diluted share, or total assets)
#fn.asr is a custom function...apply, save, remove
 
#cols for fully diluted shares #these are the denominators of the per shre items
mycol_fdshs <- match( c('cshfd','cshpri'), colnames(mydata_af.0) )  # match( c('cshfd','cshpri', 'csho'), colnames(mydata_af.0) )

mycol_prshs <- match( c('cshpri'), colnames(mydata_af.0) ) #match( c('cshpri', 'csho'), colnames(mydata_af.0) )

#df of fully diluted numerators (fully diluted per share data * coalesce(cshfd, cshpri, csho) for each period i

for(p in 1: length(mypers)) { 
i <- mypers[p]
tempfilename <- paste( "mydata_af.", i, sep='')
my_co_afnd_df <- fn.flga(tempfilename) #load file

#df of fd shares to multiply fd data by
shsfd <- coalesce( my_co_afnd_df[mycol_fdshs[1] ][,] , my_co_afnd_df[mycol_fdshs[2]][,])
shsfd_df <- data.frame(matrix(data=shsfd, nrow = nrow(my_co_afnd_df) , ncol = length(mycols_fd) ))
#head(shsfd_df)

#df of fd numerators (fd per share data * shsfd_df)
nums_fd  <- (my_co_afnd_df [mycols_fd] * (1000000*shsfd_df))/1000000
colnames(nums_fd) <- paste(colnames(mydata_af.0)[mycols_fd], '_nfd', sep='') 

########################
#df of primary shares to multiply primary data by
shspr <- coalesce( my_co_afnd_df[mycol_fdshs[1] ][,])

shspr_df <- data.frame(matrix(data=shspr, nrow = nrow(my_co_afnd_df) , ncol = length(mycols_pr) ))

#df of primary numerators (primary per share data * shspr_df)
nums_pr  <- (my_co_afnd_df [mycols_pr] * (1000000*shspr_df))/1000000
colnames(nums_pr) <- paste(colnames(mydata_af.0)[mycols_pr], '_npr', sep='') #add colname with numerator & fd or primary info

#head(my_co_afnd_df [mycols_pr])
#head(shspr_df)
#head(nums_pr)

#cbind df of numerator dfs above #maybe cbind df to co_afnd df
my_co_afnd_nums_df <- cbind (my_co_afnd_df,nums_fd ,nums_pr)

tempfilename <- paste( "mydata_af_nums.", i, sep='')
fn.asr(tempfilename , my_co_afnd_nums_df)
#rm(mydata_af1, mydata_af2, mydata_af,iname, temp, tempfilename) ; gc();gc()
} #end i
p
i
###end numerators section
